In [65]:
import json
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
from keras import regularizers
from keras import initializers

from keras.optimizers import RMSprop as rm
from keras.optimizers import Adagrad as ada
from keras.optimizers import SGD as sgd

with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches = pd.read_json(matches_json)
    
data_height = 6000
data_width = 87
label_team_width = 3
label_player_width = 29
label_pass_width = 29

mean = 0
std = 0
min_values = 0
max_values = 0

# min-max scaling
def min_max_scaling(data):
    global min_values
    global max_values
    min_values = np.amin(data, axis=(0,1), keepdims=True)
    max_values = np.amax(data, axis=(0,1), keepdims=True)
    return (data-min_values)/(max_values-min_values)

# normalizing
def normalize(data):
    global mean
    global std
    mean = np.mean(data, axis=(0,1), keepdims=True)
    std = np.std(data, axis=(0,1), keepdims=True)
    return (data - mean)/std

Small data arrays (5 matches - 4 of them train, 1 of them validation)

In [76]:
first_small_match_id = 60561
small_match_count = 5

x_small = np.ndarray(shape=(small_match_count, data_height, data_width), dtype=np.float)
y_team_small = np.ndarray(shape=(small_match_count, data_height, label_team_width), dtype=np.int)
y_player_small = np.ndarray(shape=(small_match_count, data_height, label_player_width), dtype=np.int)
y_pass_small = np.ndarray(shape=(small_match_count, data_height, label_player_width), dtype=np.int)

for id in range(small_match_count):
    x_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/x_data.npy')
    y_team_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/y_team_data.npy')
    y_player_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/y_player_data.npy')
    y_pass_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/y_pass_data.npy')

All data arrays (304 match - 243 of them train, 61 of them validation)

In [45]:
x_all = np.ndarray(shape=(len(matches['id']), data_height, data_width))
y_team_all = np.ndarray(shape=(len(matches['id']), data_height, label_team_width))
y_player_all = np.ndarray(shape=(len(matches['id']), data_height, label_player_width))
y_pass_all = np.ndarray(shape=(len(matches['id']), data_height, label_pass_width))

idx = 0
for id in matches['id']:
    x_all[idx] = np.load('../data/match_' + str(id) + '/x_data.npy')
    y_team_all[idx] = np.load('../data/match_' + str(id) + '/y_team_data.npy')
    y_player_all[idx] = np.load('../data/match_' + str(id) + '/y_player_data.npy')
    y_pass_all[idx] = np.load('../data/match_' + str(id) + '/y_player_data.npy')
    idx +=1

Normalize or scale the data

In [77]:
x_small = normalize(x_small)

with open('../model/mean_std.pkl', 'wb') as file:
    pickle.dump((mean, std), file, pickle.HIGHEST_PROTOCOL)

In [ ]:
x_small = min_max_scaling(x_small)

with open('../model/min_max.pkl', 'wb') as file:
    pickle.dump((min_values, max_values), file, pickle.HIGHEST_PROTOCOL)

In [ ]:
x_all = normalize(x_all)

with open('../model/mean_std.pkl', 'wb') as file:
    pickle.dump((mean, std), file, pickle.HIGHEST_PROTOCOL)

In [ ]:
x_all = min_max_scaling(x_all)

with open('../model/min_max.pkl', 'wb') as file:
    pickle.dump((min_values, max_values), file, pickle.HIGHEST_PROTOCOL)

Team ball possesion model

In [63]:
batch_size = 1
epochs = 100
timestep = 6000

input_dim = 87
output_dim = 3

model_team = Sequential()
model_team.add(LSTM(64, 
               input_shape=(timestep, input_dim), 
               return_sequences=True,
               dropout=0.2,
               kernel_initializer=initializers.glorot_normal(783),
               kernel_regularizer=regularizers.l2(0.01)))
model_team.add(Dense(32, kernel_initializer=initializers.glorot_normal(783), activation='relu'))
model_team.add(Dense(16, kernel_initializer=initializers.glorot_normal(783), activation='relu'))
model_team.add(Dense(output_dim, kernel_initializer=initializers.glorot_normal(783), activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=0.001, momentum=0.9, decay=1e-5, nesterov=True)

model_team.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history_team = model_team.fit(x=x_small, y=y_team_small, batch_size=batch_size, epochs=epochs, verbose=2, 
                              callbacks=None, validation_split=0.2, validation_data=None, shuffle=True, 
                              class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, 
                              validation_steps=None)

Train on 4 samples, validate on 1 samples
Epoch 1/100


 - 29s - loss: 2.0820 - acc: 0.3468 - val_loss: 1.3046 - val_acc: 0.3723


Epoch 2/100


 - 27s - loss: 1.7171 - acc: 0.3595 - val_loss: 1.2827 - val_acc: 0.3780


Epoch 3/100


...

 - 28s - loss: nan - acc: 0.4705 - val_loss: nan - val_acc: 0.5407


Epoch 99/100


 - 28s - loss: nan - acc: 0.4705 - val_loss: nan - val_acc: 0.5407


Epoch 100/100


 - 28s - loss: nan - acc: 0.4705 - val_loss: nan - val_acc: 0.5407


In [64]:
# summarize history for accuracy
plt.plot(history_team.history['acc'])
plt.plot(history_team.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../graphs/team_acc.png')
plt.show()

# summarize history for loss
plt.plot(history_team.history['loss'])
plt.plot(history_team.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../graphs/team_loss.png')
plt.show()

In [61]:
model_team.save('../model/team_model.h5')
model_team.save_weights('../model/team_model_weight.h5')
json_data = model_team.to_json()

with open('../model/team_json_data.txt', 'w') as outfile:
    json.dump(json_data, outfile)

Player ball possesion model

In [81]:
x_small = np.concatenate((x_small, y_team_small), axis=2)

In [ ]:
x_all = np.concatenate((x_all, y_team_all), axis=2)

In [84]:
batch_size = 1
epochs = 100
timestep = 6000

input_dim = 90
output_dim = 29

model_player = Sequential()
model_player.add(LSTM(72, 
               input_shape=(timestep, input_dim), 
               return_sequences=True,
               dropout=0.2,
               #activation='relu',
               kernel_initializer=initializers.glorot_normal(783),
               kernel_regularizer=regularizers.l2(0.01)))
model_player.add(Dense(36, kernel_initializer=initializers.glorot_normal(783), activation='relu'))
model_player.add(Dense(output_dim, kernel_initializer=initializers.glorot_normal(783), activation='softmax'))

#opt = rm()
opt = ada()
#opt = sgd(lr=0.001, momentum=0.9, decay=1e-5, nesterov=True)

model_player.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history_player = model_player.fit(x=x_small, y=y_player_small, batch_size=batch_size, epochs=epochs, verbose=2, 
                                  callbacks=None, validation_split=0.2, validation_data=None, shuffle=True, 
                                  class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, 
                                  validation_steps=None)

Train on 4 samples, validate on 1 samples
Epoch 1/100


 - 30s - loss: 3.9764 - acc: 0.1414 - val_loss: 3.6607 - val_acc: 0.3103


Epoch 2/100


 - 27s - loss: 3.2855 - acc: 0.4395 - val_loss: 3.2443 - val_acc: 0.3603


Epoch 3/100


...

 - 26s - loss: 1.4738 - acc: 0.5942 - val_loss: 1.9735 - val_acc: 0.5422


Epoch 99/100


 - 27s - loss: 1.4899 - acc: 0.5847 - val_loss: 1.9814 - val_acc: 0.5462


Epoch 100/100


 - 27s - loss: 1.7046 - acc: 0.5502 - val_loss: 1.9596 - val_acc: 0.5492


In [85]:
# summarize history for accuracy
plt.plot(history_player.history['acc'])
plt.plot(history_player.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../graphs/player_acc.png')
plt.show()

# summarize history for loss
plt.plot(history_player.history['loss'])
plt.plot(history_player.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../graphs/player_loss.png')
plt.show()

In [86]:
model_player.save('../model/player_model.h5')
model_player.save_weights('../model/player_model_weight.h5')
json_data = model_player.to_json()

with open('../model/player_json_data.txt', 'w') as outfile:
    json.dump(json_data, outfile)

Pass model

In [87]:
x_small = np.concatenate((x_small, y_player_small), axis=2)

In [ ]:
x_all = np.concatenate((x_all, y_player_all), axis=2)

In [92]:
batch_size = 1
epochs = 100
timestep = 6000

input_dim = 119
output_dim = 29

model_pass = Sequential()
model_pass.add(LSTM(96, 
               input_shape=(timestep, input_dim), 
               return_sequences=True,
               kernel_initializer=initializers.glorot_normal(783),
               kernel_regularizer=regularizers.l2(0.01)))
model_pass.add(Dropout(0.2))
model_pass.add(Dense(64, kernel_initializer=initializers.glorot_normal(783), activation='relu'))
model_pass.add(Dense(output_dim, kernel_initializer=initializers.glorot_normal(783), activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=0.001, momentum=0.9, decay=1e-5, nesterov=True)

model_pass.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history_pass = model_pass.fit(x=x_small, y=y_pass_small, batch_size=batch_size, epochs=epochs, verbose=2, 
                                  callbacks=None, validation_split=0.2, validation_data=None, shuffle=True, 
                                  class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, 
                                  validation_steps=None)

Train on 4 samples, validate on 1 samples
Epoch 1/100


 - 29s - loss: 1.9156 - acc: 0.0157 - val_loss: 1.8177 - val_acc: 0.0062


Epoch 2/100


 - 24s - loss: 1.9148 - acc: 0.0180 - val_loss: 1.8165 - val_acc: 0.0062


Epoch 3/100


...

 - 32s - loss: 1.7128 - acc: 0.0165 - val_loss: 1.6172 - val_acc: 0.0060


Epoch 99/100


 - 29s - loss: 1.7106 - acc: 0.0173 - val_loss: 1.6153 - val_acc: 0.0060


Epoch 100/100


 - 28s - loss: 1.7090 - acc: 0.0164 - val_loss: 1.6133 - val_acc: 0.0060


In [93]:
# summarize history for accuracy
plt.plot(history_pass.history['acc'])
plt.plot(history_pass.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../graphs/pass_acc.png')
plt.show()

# summarize history for loss
plt.plot(history_pass.history['loss'])
plt.plot(history_pass.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../graphs/pass_loss.png')
plt.show()

In [90]:
model_pass.save('../model/pass_model.h5')
model_pass.save_weights('../model/pass_model_weight.h5')
json_data = model_pass.to_json()

with open('../model/pass_json_data.txt', 'w') as outfile:
    json.dump(json_data, outfile)